Wedge Part 3

In [2]:
import os
import re
import datetime 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_gbq
import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

from os.path import exists


Establishing a connection with GBQ

In [3]:
# These first two values will be different on your machine. 
service_path = "/Users/willd/Desktop/ADA/"
service_file = 'msba-wd-7a44c8ddeff2.json' # change this to your authentication information  
gbq_proj_id = 'msba-wd' # 'msba-wd' # change this to your project. 

# And this should stay the same. 
private_key =service_path + service_file

print(exists(private_key))

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

print(credentials)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

for item in client.list_datasets() : 
    print(item.full_dataset_id)

True
msba-wd:class_example
msba-wd:dram_shop
msba-wd:wedge_proj


Below are 3 queried tables made into a pandas dataframe.

1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items .

In [286]:
query = """
SELECT 
EXTRACT(date FROM (CAST(datetime AS TIMESTAMP))) AS date
, EXTRACT(hour FROM (CAST(datetime AS TIMESTAMP))) AS hour
, ROUND(SUM(total),2) AS total_spend
, COUNT(DISTINCT CONCAT 
(CAST (register_no AS STRING), CAST(emp_no AS STRING), CAST(trans_no AS STRING),CAST(EXTRACT(date FROM (CAST(datetime AS TIMESTAMP)))AS STRING))) AS total_trans
, SUM( CASE WHEN trans_status IN ('V', 'R') THEN -1 ELSE 1 END) AS items
FROM `msba-wd.wedge_proj.transArchive*`
WHERE department NOT IN (0,15)
AND (trans_status is NULL or trans_status in (' ', 'V', 'R'))
GROUP BY date, hour
ORDER BY date DESC, hour;


"""

sales_data = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)
sales_data.head()


Downloading: 100%|█████████| 37907/37907 [00:02<00:00, 17623.93rows/s]


,date,hour,total_spend,total_trans,items
0,2016-12-31,7,43.40,4,14
1,2016-12-31,8,2143.15,65,488
2,2016-12-31,9,5877.28,119,1279
3,2016-12-31,10,7639.51,190,1772
4,2016-12-31,11,9172.63,239,2226


2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items. 

In [4]:
query = """
SELECT 
  DISTINCT(card_no) AS card_no
  , EXTRACT(year FROM (CAST(datetime AS TIMESTAMP))) AS year
  , EXTRACT(month FROM (CAST(datetime AS TIMESTAMP))) AS month
  -- EXTRACT(day FROM datetime) AS day
  , SUM(total) AS sales 
  , COUNT( DISTINCT
          CONCAT(
            CAST(EXTRACT(DATE FROM(CAST(datetime AS TIMESTAMP))) AS STRING), 
            CAST(register_no AS STRING ),
            CAST(emp_no AS STRING),
            CAST(trans_no AS STRING)
        ) ) AS transactions
,SUM(CASE WHEN trans_status in ('V', 'R') THEN -1 ELSE 1 END) AS items
FROM `msba-wd.wedge_proj.transArchive*`
GROUP BY card_no, year, month
ORDER BY year, month;
"""

owner_data = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)
owner_data.head()


Downloading: 100%|███████| 762927/762927 [00:34<00:00, 21845.96rows/s]


,card_no,year,month,sales,transactions,items
0,21125,2010,1,2.398082e-13,14,203
1,12388,2010,1,-1.740830e-13,11,143
2,24128,2010,1,-1.865175e-14,14,165
3,17162,2010,1,3.237410e-13,14,197
4,19180,2010,1,4.201084e-13,14,277


3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [287]:
query = """
SELECT 
  trans.upc
  ,trans.description
  ,trans.department as dept_number
  ,Dept.dept_name
  , EXTRACT(YEAR from (CAST(trans.datetime AS TIMESTAMP))) AS year
  , EXTRACT(MONTH FROM (CAST(trans.datetime AS TIMESTAMP))) AS month
  ,ROUND(SUM(total),2) AS sales
  ,COUNT( DISTINCT
          CONCAT(
            CAST(EXTRACT(DATE FROM (CAST(datetime AS TIMESTAMP))) AS STRING), 
            CAST(register_no AS STRING ),
            CAST(emp_no AS STRING),
            CAST(trans_no AS STRING)
        ) ) AS transactions
  ,SUM(CASE WHEN trans_status in ('V', 'R') THEN -1 ELSE 1 END) AS items
FROM `msba-wd.wedge_proj.transArchive*` AS trans
JOIN `umt-msba.wedge_example.departments` AS Dept
ON Dept.department = trans.department 
GROUP BY description, upc, year, month, trans.department, Dept.dept_name;
"""

owner_data = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)
owner_data.head()


Downloading: 100%|█████| 1152682/1152682 [01:34<00:00, 12199.71rows/s]


,upc,description,dept_number,dept_name,year,month,sales,transactions,items
0,9.69DP16,FISH&SEAFOOD,16,FISH&SEAFOOD,2014,5,9.69,1,1
1,0000000004205,WINE Glass Zuccardi a Malbec,22,BEER & WINE,2015,5,16.00,1,1
2,0020685800000,Lake Trout,16,FISH&SEAFOOD,2015,3,1427.68,159,164
3,4.81DP16,FISH&SEAFOOD,16,FISH&SEAFOOD,2016,8,4.81,1,1
4,3.50DP16,FISH&SEAFOOD,16,FISH&SEAFOOD,2012,9,3.50,1,1
